In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.figure_factory as ff

In [13]:
data = pd.read_csv('hcvdat0.csv')

In [14]:
values={'0=Blood Donor':'BloodDonor', '0s=suspect Blood Donor':'suspect Blood Donor', '1=Hepatitis':'Hepatitis', '2=Fibrosis':'Fibrosis', '3=Cirrhosis':'Cirrhosis'}

In [15]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,615.0,308.000000,177.679487,1.00,154.500,308.00,461.500,615.00
Age,615.0,47.408130,10.055105,19.00,39.000,47.00,54.000,77.00
ALB,614.0,41.620195,5.780629,14.90,38.800,41.95,45.200,82.20
ALP,597.0,68.283920,26.028315,11.30,52.500,66.20,80.100,416.60
ALT,614.0,28.450814,25.469689,0.90,16.400,23.00,33.075,325.30
AST,615.0,34.786341,33.090690,10.60,21.600,25.90,32.900,324.00
BIL,615.0,11.396748,19.673150,0.80,5.300,7.30,11.200,254.00
CHE,615.0,8.196634,2.205657,1.42,6.935,8.26,9.590,16.41
CHOL,605.0,5.368099,1.132728,1.43,4.610,5.30,6.060,9.67
CREA,615.0,81.287805,49.756166,8.00,67.000,77.00,88.000,1079.10


In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
category = le.fit_transform(data.Category)

sex = le.fit_transform(data.Sex)

data['sex'] = sex 
data['category'] = category
data = data.drop('Sex',axis=1)
data = data.drop('Category',axis=1)
data = data.drop('Unnamed: 0',axis=True)

table = ff.create_table(data.head(-5))
py.iplot(table, filename='dataframe_ex_preview')

In [17]:

MeanValues = {
          'ALB':data.ALB.mean() , 
          'ALP': data.ALP.mean(), 
          'ALT': data.ALT.mean(), 
          'CHOL': data.CHOL.mean(),
          'PROT': data.PROT.mean(),           
         }

In [18]:
data = data.fillna(value = MeanValues)
data.head()

,Age,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,sex,category
0,32,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0,1,0
1,32,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5,1,0
2,32,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,1,0
3,32,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,1,0
4,32,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7,1,0


In [19]:
data.isnull().sum()

Age         0
ALB         0
ALP         0
ALT         0
AST         0
BIL         0
CHE         0
CHOL        0
CREA        0
GGT         0
PROT        0
sex         0
category    0
dtype: int64

In [22]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

X = data.iloc[:, 0:12]
Y = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33, random_state=30)

In [23]:
from sklearn.neural_network import MLPClassifier
ysa = MLPClassifier()


pipeline_4 = Pipeline([('scaler', StandardScaler()), ('ysa', ysa)])
pipeline_4

Pipeline(steps=[('scaler', StandardScaler()), ('ysa', MLPClassifier())])

In [26]:
parameteres_ysa = {
    'ysa__hidden_layer_sizes':[
        (11,8,5,5,3),
        (25,4,2),
        (50,10,5,2),
        (50,25,10,5),
        (50,35,20,10,3)
                              ],
    'ysa__activation': ['tanh', 'relu','logistic','identity'],
    'ysa__solver': ['sgd', 'adam','lbfgs'],
    'ysa__alpha': [0.0001, 0.05,0.1,0.001],
    'ysa__learning_rate': ['constant','adaptive','invscaling']
}
# Grdisearch
grid = GridSearchCV(pipeline_4, param_grid=parameteres_ysa, cv=5,n_jobs=-1)
grid.fit(X_train, y_train)
grid

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('ysa', MLPClassifier())]),
             n_jobs=-1,
             param_grid={'ysa__activation': ['tanh', 'relu', 'logistic',
                                             'identity'],
                         'ysa__alpha': [0.0001, 0.05, 0.1, 0.001],
                         'ysa__hidden_layer_sizes': [(11, 8, 5, 5, 3),
                                                     (25, 4, 2), (50, 10, 5, 2),
                                                     (50, 25, 10, 5),
                                                     (50, 35, 20, 10, 3)],
                         'ysa__learning_rate': ['constant', 'adaptive',
                                                'invscaling'],
                         'ysa__solver': ['sgd', 'adam', 'lbfgs']})

In [30]:
print("score = %3.4f" %(grid.score(X_test,y_test)))
print(grid.best_params_)


score = 0.9113
{'ysa__activation': 'tanh', 'ysa__alpha': 0.1, 'ysa__hidden_layer_sizes': (50, 25, 10, 5), 'ysa__learning_rate': 'adaptive', 'ysa__solver': 'lbfgs'}
